<style>
.text_cell_render {
font-family: Times, serif;
}
</style>


# Classification of black vs. death metal using ML

Not long ago I stumbled upon a very interesting blog post:

https://www.degeneratestate.org/posts/2016/Apr/20/heavy-metal-and-natural-language-processing-part-1/

where NLP techniques were used to gain insight into the corpus of heavy metal lyrics.  One of the things that caught my eye was the [dendrogram](https://www.degeneratestate.org/posts/2016/Apr/20/heavy-metal-and-natural-language-processing-part-1/cluster_dendrogram.png) obtained via hierarchical clustering of the lyrics of the most popular bands.  To quote the blogpost:

> In most cases the clustering does a good job of capturing similarity between bands. Ozzy Osborne and Black Sabbath end up close to each other, as do Rainbow and Dio. There is a clear Power Metal grouping, as well as various other small groupings that make vague sense: Madball is next to Agnostic Front; Darkthrone, Dimmu Borgir and Emperor are all close together; Cannibal Corpse and Carcass wallow together in gore.

Indeed, the clustering did quite well in finding an extreme metal cluster and it further distinguishes between black metal and death metal.  What I found surprising at the time was the fact that the black metal band Mayhem was actually deemed closer lyrically to death metal - so I decided to take a closer look into that.

Thus the goal of this project was to explore to what extent simple ML algorithms can distinguish between death and black metal based exclusively on the lyrics.  While this is in and of itself an interesting question, it is of course also a simplifying assumption and I would be curious to see how this would extend to a multi-label classification problem for more genre of metal or non-metal lyrics.

Starting out, I expected simple ML algorithms to do OK, but would be very surprised if it were to attain more than an 80% accuracy score.  After all, the two extreme metal genres have considerable thematical and historical overlap, the style of the bands may significantly change with time, and there are also non-negligible influences from other music genres, all of this making the two groups fairly heterogeneous.  However both bands and metal fans agree that there are two distinct genres and we all expect that a song with a lot of gore and decapitation is most likely death metal, while a song about freezing moons and sacrifices to Satan most likely black metal, so there is definitely hope for success.


## The data
The first step in the project was of course data collection.  Since there is no "official" labelled heavy metal dataset I made my own using a scrape of the darklyrics website and a judicious choice of bands for each genre.  To be precise, I wanted to consider the most "influential" bands for each genre and the chosen measure of that was their presence in the wikipedia articles for [death metal](https://en.wikipedia.org/wiki/Death_metal) and [black metal](https://en.wikipedia.org/wiki/Black_metal), respectively.  Even so, I have not included every band mentioned in those articles in order to keep the two groups balanced especially given the penchant of black metal bands to use languages other than English.

Due to copyright concerns I have chosen not to make the lyrics dataset available.  I have however uploaded two `csv` files containing the names of the bands for each of the two genres together with the number of songs for each band.

In [1]:
import pandas as pd

df = pd.read_csv('metal_data_final.csv', encoding='utf-8')

## Dataframe characteristics

Let's take a look at the dataframe.  The genre labels are 0 for death and 1 for black metal.  The lyrics have been preprocessed to be all lower case and all punctuation has been ignored.  All the lyrics are in English.

In [2]:
df.head(10)

,lyrics,band,genre
0,oh how my rampant desire ripped from the pres...,cradle of filth,1
1,martyrs of the cross bear the flames of hate t...,immolation,0
2,in the cold twilight i stand watching the rise...,dark funeral,1
3,even now where i adjoin the wick architeuthis ...,bethlehem,1
4,insatiable lust for a further deprivation an e...,dyscarnate,0
5,here in the garden we know not what we do made...,cattle decapitation,0
6,lechery on the altar we burn in blaspemy orgas...,belphegor,1
7,i dream of leaving this body through a leak in...,pig destroyer,0
8,all they hold sacred all they consecrated corr...,belphegor,1
9,wasted ashore by a stormfull flood left behind...,vreid,1


Note that all songs of a given band are labelled with the same genre.

In [3]:
number_death_songs = song_number_death = df[df['genre'] == 0].shape[0]
number_black_songs = song_number_death = df[df['genre'] == 1].shape[0]

number_death_bands = df[df['genre'] == 0]['band'].value_counts().shape[0]
number_black_bands = df[df['genre'] == 1]['band'].value_counts().shape[0]

print('Total number of songs:', len(df))
print('Total number of black metal songs', number_black_songs)
print('Total number of death metal songs', number_death_songs)
print('Total number of black metal bands', number_black_bands)
print('Total number of death metal bands', number_death_bands)

Total number of songs: 8991
Total number of black metal songs 4367
Total number of death metal songs 4624
Total number of black metal bands 89
Total number of death metal bands 57


## Metal topics

Let's first use LDA to see if death and black metal songs can be reasonably grouped topicwise.

In [4]:
from sklearn.feature_extraction import text 
my_additional_stop_words = ['ov', 'oh', 'thy', 'thou', 'thee', 'don', 've', 't']
stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

In [11]:
from metal_lda import metal_lda

lda, count = metal_lda(df, stop_words, 5)

In [12]:
n_top_words = 10
feature_names = count.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i] for i in topic.argsort() [:-n_top_words - 1:-1]]))

Topic 1:
war satan evil holy lord christ burn rise heaven cross
Topic 2:
land sun hear long wind old far place past sea
Topic 3:
kill want make fucking live need head body fuck won
Topic 4:
dreams forever shadows eternal deep souls human burning rise chaos
Topic 5:
new metal gods sun stand power long fight truth faith


### Black metal topics

In [13]:
df_black = df[df['genre'] == 1]
lda, count = metal_lda(df_black, stop_words, 3)

In [14]:
n_top_words = 15
feature_names = count.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i] for i in topic.argsort() [:-n_top_words - 1:-1]]))

Topic 1:
want inside long lost did left place dream say truth leave hand need live new
Topic 2:
moon wind shadows winds rise ancient flames land stars souls wings gods red deep high
Topic 3:
satan burn holy kill christ lord power fucking master fight gods sin born son human


### Death metal topics

In [16]:
df_death = df[df['genre'] == 0]
lda, count = metal_lda(df_death, stop_words, 3)

In [17]:
n_top_words = 15
feature_names = count.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i] for i in topic.argsort() [:-n_top_words - 1:-1]]))

Topic 1:
skin grave cut corpse bones brain open rot alive bodies rotting fucking evil knife fuck
Topic 2:
shall rise darkness war dark burn souls forever eternal evil gods new sky burning sun
Topic 3:
won right need want say cause fucking truth think words look tell believe control hear


## Logistic Regression

Can a simple logistic regression ML algorithm distinguish between death and black metal?

In [4]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.33, stratify=df['genre'], random_state=1)

In [5]:
X_train = df_train['lyrics'].values
y_train = df_train['genre'].values

X_test = df_test['lyrics'].values
y_test = df_test['genre'].values

### Grid search

Let's see what we get - focus on 3- and 4- grams.

In [7]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

stop = stopwords.words('english')
def tokenizer(text):
    return text.split()

porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

param_grid = [{'vect__ngram_range': [(1,3), (1,4)],
    'vect__stop_words': [stop, None],
    'vect__tokenizer': [tokenizer, tokenizer_porter],
    'clf__penalty': ['l1', 'l2'],
    'clf__C': [1.0, 10.0, 100.0]}]
                 
lr_tfidf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=0, solver='liblinear'))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5, verbose=2,
                           n_jobs=1)

In [25]:
# run the fit
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] END clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 3), vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 

/Users/mara/PycharmProjects/metal/venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 3), vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'o

/Users/mara/PycharmProjects/metal/venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 3), vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'o

...

/Users/mara/PycharmProjects/metal/venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  warnings.warn(


[CV] END clf__C=100.0, clf__penalty=l2, vect__ngram_range=(1, 4), vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 

/Users/mara/PycharmProjects/metal/venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  warnings.warn(


[CV] END clf__C=100.0, clf__penalty=l2, vect__ngram_range=(1, 4), vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0,
                                                           solver='liblinear'))]),
             n_jobs=1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 3), (1, 4)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                                'yours', 'yourself',
                                                'yourselves', 'he', 'him',
  

In [26]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)

Best parameter set: {'clf__C': 100.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 3), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer_porter at 0x7ff9b6a18280>} 


In [27]:
print('CV Accuracy: %.3f'
      % gs_lr_tfidf.best_score_)

CV Accuracy: 0.769


In [28]:
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f'
      % clf.score(X_test, y_test))

Test Accuracy: 0.780


In [36]:
from joblib import dump, load
dump(clf, 'metal-class.joblib')

['metal-class.joblib']

In [7]:
from joblib import load
clf = load('metal-class.joblib')

### Analysis

Not a bad estimate, more or less as expected.  Let's see some statistics and whether the songs that were wrongly labelled by the estimator would be confusing for a human as well.

First of all, let's see for each genre what are the most influential bands in the training set, i.e. the bands with the most songs.  We expect that their songs would be labelled correctly and would also score high in their respective genre.

Let's first get a dataframe containing all the bands in the training set, their genre, and the number of their songs.

In [8]:
import analysis

df_train_bands = analysis.bands_songs_genre(df_train)
df_train_bands.head(10)

,band,songs,genre
0,abbath,16,1
1,aborym,33,1
2,abscess,73,0
3,agalloch,25,1
4,akercocke,39,1
5,anaal nathrakh,31,0
6,ancient,35,1
7,ancient rites,47,1
8,andoceans,1,1
9,arallu,23,1


So let's see now for each genre the top 10 most influential bands

In [9]:
df_train_grouped = df_train_bands.groupby('genre')

In [37]:
df_train_grouped.get_group(1).sort_values(by='songs', ascending=False).head(10)

,band,songs,genre
40,cradle of filth,129,1
82,impaled nazarene,102,1
17,bathory,96,1
89,marduk,92,1
46,darkthrone,88,1
18,behemoth,82,1
120,samael,80,1
117,rotting christ,79,1
64,enslaved,64,1
55,dimmu borgir,61,1


In [11]:
df_train_grouped.get_group(0).sort_values(by='songs', ascending=False).head(10)

,band,songs,genre
103,napalm death,200,0
123,sepultura,137,0
125,six feet under,117,0
44,dark tranquillity,115,0
31,cannibal corpse,110,0
95,mortification,110,0
84,in flames,110,0
129,soilwork,101,0
112,pig destroyer,85,0
137,unleashed,84,0


The expectation is therefore that these bands have a good guess ratio for their songs in the test dataset.  So let's look at the wrong vs. correct predictions of our classifier.  I am also interested to see the probability scores that the classifier assigns for each guess so let's first augment the test dataframe to reflect this information as well.

In [12]:
# reset indices for df_train and df_test
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [13]:
analysis.add_predicted_proba(clf, df_test, X_test);

In [14]:
df_wrong_predictions, df_correct_predictions = analysis.wrong_predictions(clf, df_test, X_test, y_test)

In [15]:
df_wrong_predictions.head(5)

,lyrics,band,genre,probabilities
3,i m chaos international the writing on the wal...,cradle of filth,1,"[0.6467133914410897, 0.35328660855891025]"
10,i asset my claim of uncompromising empowerment...,nile,0,"[0.02354249812638476, 0.9764575018736152]"
13,traced in a wet sand her name in perfect cursi...,pig destroyer,0,"[0.35526107010205554, 0.6447389298979445]"
20,year upon year month after month consumed and ...,ancient,1,"[0.7135057772765233, 0.2864942227234767]"
32,on my throne of sin i watch the demons feed n...,dismember,0,"[0.4635622729553849, 0.5364377270446151]"


In [16]:
df_correct_predictions.head(5)

,lyrics,band,genre,probabilities
0,i mean no disrespect how can you say what you ...,salt the wound,0,"[0.9513753382976569, 0.04862466170234313]"
1,cabal disguised divided not ruled by hidden ha...,mayhem,1,"[0.36414499061481187, 0.6358550093851881]"
2,night gleaming moon the hours passing fast int...,bloodbath,0,"[0.8128124262326701, 0.18718757376732986]"
4,i survived time and death to meet you to be su...,mutiilation,1,"[0.27350214764956815, 0.7264978523504318]"
5,its cold on the outskirts of the fire and we s...,panopticon,1,"[0.4355700827291118, 0.5644299172708882]"


So let's see the proportion of wrong guesses per band:

In [17]:
df_bands = analysis.bands(df_test)
df_wrong_predictions_perband = analysis.wrong_predictions_perband(df_wrong_predictions, df_bands)
df_wrong_predictions_perband.head(20)

,band,wrong_predictions,songs,wrong_guesses_percentage
29,cattle decapitation,1,48,2.083333
14,behemoth,1,46,2.173913
83,napalm death,3,88,3.409091
24,cannibal corpse,2,57,3.508772
99,six feet under,2,51,3.921569
50,emperor,1,25,4.000000
40,death,1,25,4.000000
37,darkane,1,23,4.347826
110,watain,1,21,4.761905
105,suffocation,1,20,5.000000


So we see that indeed some top 10 bands in each genre have a pretty low wrong guess percentage (under 10%):
  * *Napalm Death, Cannibal Corpse, Six Feet Under, Soilwork* for death metal
  * *Marduk, Behemoth, Cradle of Filth* for black metal

but interestingly, there are also bands represented by (in some cases significantly) fewer songs which are also correctly labelled most of the time:
  * *Cattle Decapitation, Death, Suffocation, Bolt Thrower, Skinless* for death metal
  * *Emperor, Darkane, Watain, Dark Funeral* for black metal

### The case of Mayhem
Let's take a close look at one of the most influential black metal bands, *mayhem*.  First of all it has a pretty high percentage of wrong guesses on the test data set:

In [18]:
df_wrong_predictions_perband[df_wrong_predictions_perband['band'] == 'mayhem']

,band,wrong_predictions,songs,wrong_guesses_percentage
74,mayhem,11,30,36.666667


So let's see what kind of *Mayhem* songs are not guesses correctly:

In [19]:
df_wrong_predictions[df_wrong_predictions['band'] == 'mayhem'].head(11)

,lyrics,band,genre,probabilities
78,a face in stone decayed by age a man who has r...,mayhem,1,"[0.5782474708535404, 0.42175252914645955]"
369,fight we will fight right living low in a wor...,mayhem,1,"[0.617218347120128, 0.3827816528798719]"
479,penetrate the lions at the gates illusions flo...,mayhem,1,"[0.5411187089354885, 0.45888129106451153]"
696,he stood there so many times funerals get orga...,mayhem,1,"[0.6287726340006508, 0.3712273659993493]"
819,to creature give life create a primitive worke...,mayhem,1,"[0.6429524040491512, 0.35704759595084873]"
1044,a dream of another existence you wish to die a...,mayhem,1,"[0.7566548075587995, 0.24334519244120043]"
1062,i remember the future a new beginning of time,mayhem,1,"[0.5952601520417331, 0.4047398479582669]"
1371,enter now my palace of sacrificial guilt on th...,mayhem,1,"[0.5253407296528971, 0.47465927034710287]"
2143,in the century where man dies we the huntress ...,mayhem,1,"[0.953897639992351, 0.04610236000764895]"
2664,a face in stone decayed by age a man who has r...,mayhem,1,"[0.6328084910347173, 0.36719150896528274]"


So we see that mostly the classifier is not very sure which genre this songs are.  It is however very sure that the song below (Bad Blood from the album Daemon) is a death metal song:

In [20]:
df_wrong_predictions[df_wrong_predictions['band'] == 'mayhem']['lyrics'][2837]

'so much flowing bad blood rising like the flood there is no cure for what you have nothing can fix it you poisoned your own blood there is no coming back no way out you are possessed face your hopeless fate prodigal bad blood brotherhood destroyed i won t forgive i won t forget i will take my revenge you crossed my red line i sentence you to death when the time comes i will have you killed die now bad blood open your veins drain out all the filth within open your veins i won t forgive i won t forget bad blood die now die now you crossed my red line i sentence you to death when the time comes i will have you killed die now bad blood '

It has a lot of blood and violence...makes sense.  What about the correct predictions?

In [21]:
df_correct_predictions[df_correct_predictions['band'] == 'mayhem'].head(10)

,lyrics,band,genre,probabilities
1,cabal disguised divided not ruled by hidden ha...,mayhem,1,"[0.36414499061481187, 0.6358550093851881]"
61,caress the deceitful snake the one who speaks ...,mayhem,1,"[0.4628211555319124, 0.5371788444680876]"
157,witchcraft blood and satan meet the face of de...,mayhem,1,"[0.005759631690151701, 0.9942403683098483]"
193,demonic laughter your cremation your lungs gas...,mayhem,1,"[0.0498321600706082, 0.9501678399293918]"
554,come hear the moon is calling the witching hou...,mayhem,1,"[0.002650301020071022, 0.997349698979929]"
588,you are blind deny the sun and light whose ar...,mayhem,1,"[0.06921435612411797, 0.930785643875882]"
852,every time this year this dark fog will appear...,mayhem,1,"[0.017838133184150284, 0.9821618668158497]"
872,do you hear with eyes the universe is crunchin...,mayhem,1,"[0.40688827078307865, 0.5931117292169213]"
1166,black is the night metal we fight power amps ...,mayhem,1,"[0.00982915969383058, 0.9901708403061694]"
1318,from these maleficent realms invoke the oath u...,mayhem,1,"[0.38483457091127926, 0.6151654290887207]"


Feel more occulty, also makes sense.  There are a few bands that are completely correctly classified (28 to be precise), for example the black metal band *Immortal* (18 songs in the test dataframe):

In [22]:
print(len(df_correct_predictions[df_correct_predictions['band'] == 'immortal']) == 
      len(df_test[df_test['band'] == 'immortal']))

True


My guess would be then that *Immortal* songs are scored very high on black metalness, i.e. they should have numbers close to 1 as second entries in the `probabilities` tuples.  Indeed, look at the difference between *Mayhem* and *Immortal*:

In [23]:
print('Average black metalness of Immortal test songs: %.3f '
      % analysis.average_blackness(df_correct_predictions[df_correct_predictions['band'] == 'immortal']))
print('Average black metalness of correctly labelled Mayhem songs: %.3f '
      % analysis.average_blackness(df_correct_predictions[df_correct_predictions['band'] == 'mayhem']))
print('Average black metalness of wrongly labelled Mayhem songs: %.3f '
      % analysis.average_blackness(df_wrong_predictions[df_wrong_predictions['band'] == 'mayhem']))
print('Average black metalness of all Mayhem test songs: %.3f '
      % analysis.average_blackness(df_test[df_test['band'] == 'mayhem']))


Average black metalness of Immortal songs: 0.953 
Average black metalness of correctly labelled Mayhem songs: 0.819 
Average black metalness of wrongly labelled Mayhem songs: 0.329 
Average black metalness of all Mayhem songs: 0.639 


For comparison, let's also take a quick look on the death metal side and consider the tech death band *Psycroptic* (16 songs in the test dataframe, comparable to *Immortal*'s 18) where we see a similar behaviour:

In [24]:
print(len(df_correct_predictions[df_correct_predictions['band'] == 'psycroptic']) == 
      len(df_test[df_test['band'] == 'psycroptic']))

True


In [25]:
print('Average death metalness of Psycroptic songs: %.3f '
      % analysis.average_deathness(df_correct_predictions[df_correct_predictions['band'] == 'psycroptic']))

Average death metalness of Psycroptic songs: 0.900 


So the same phenomenon occurs.  On the other hand, I would expect death metal bands like *Dark Tranquillity, Morbid Angel* or *Nile* to also have relatively high wrong prediction percentages due to their thematic difference to bands like *Cannibal Corpse* or *Napalm Death*.  Indeed:

In [28]:
df_wrong_predictions_perband[df_wrong_predictions_perband['band'].isin(
    ['dark tranquillity',
     'morbid angel',
     'nile'])]

,band,wrong_predictions,songs,wrong_guesses_percentage
36,dark tranquillity,9,48,18.750000
76,morbid angel,13,34,38.235294
87,nile,17,33,51.515152


and here are their average death metal scores taken over the whole test dataset:

In [30]:
print('Average death metalness of all Dark Tranquillity test songs: %.3f '
      % analysis.average_deathness(df_test[df_test['band'] == 'dark tranquillity']))
print('Average death metalness of all Morbid Angel test songs: %.3f '
      % analysis.average_deathness(df_test[df_test['band'] == 'morbid angel']))
print('Average death metalness of all Nile test songs: %.3f '
      % analysis.average_deathness(df_test[df_test['band'] == 'nile']))


Average death metalness of all Dark Tranquillity test songs: 0.740 
Average death metalness of all Morbid Angel test songs: 0.620 
Average death metalness of all Nile test songs: 0.501 


and over the correct guesses:

In [31]:
print('Average death metalness of correctly labelled Dark Tranquillity songs: %.3f '
      % analysis.average_deathness(df_correct_predictions[df_correct_predictions['band'] == 'dark tranquillity']))
print('Average death metalness of correctly labelled Morbid Angel songs: %.3f '
      % analysis.average_deathness(df_correct_predictions[df_correct_predictions['band'] == 'morbid angel']))
print('Average death metalness of correctly labelled Nile songs: %.3f '
      % analysis.average_deathness(df_correct_predictions[df_correct_predictions['band'] == 'nile']))

Average death metalness of correctly labelled Dark Tranquillity songs: 0.853 
Average death metalness of correctly labelled Morbid Angel songs: 0.863 
Average death metalness of correctly labelled Nile songs: 0.815 


This is interesting - *Morbid Angel* has very similar statistics to *Mayhem*, while *Dark Tranquillity* seems to do better.  However we can also see that *Nile* is similar to *Mayhem* in having an average death metal score of about 81-82%, but it has a significantly lower score when the average is taken over the whole of the data set: it's 50% compared to *Mayhem's* 64%.  So lyrically *Nile* is almost a black metal band according to our classifier, while *Mayhem* is mostly a black metal band, with some (a lot) of death sprinkled in. 

This fits with the history and also with the observation from the blogpost where *Mayhem* appeared closer to *Bolt Thrower* than to *Behemoth*.

Finally, let's see what are the top 10 most black/death metal (correctly labelled) songs according to our classifier.  

In [33]:
df_correct_predictions_sorted = df_correct_predictions.sort_values(by=['probabilities'])
df_correct_predictions_sorted.head(10)

,lyrics,band,genre,probabilities
2589,as the twilight gave birth to the night i walk...,naglfar,1,"[1.5662292068507888e-05, 0.9999843377079315]"
1839,a sorcery written in blood whispered by a witc...,bathory,1,"[1.981708038301111e-05, 0.999980182919617]"
1287,wings of satan orb my heart it burns with love...,watain,1,"[2.4684499169702612e-05, 0.9999753155008303]"
2787,i believe in satan who rend both heavens and ...,behemoth,1,"[6.667012995287269e-05, 0.9999333298700471]"
1588,reborn through the breeze in a snowstorm in fr...,enthroned,1,"[8.336662506347459e-05, 0.9999166333749365]"
194,i hear the evil calling of the spirits of the ...,marduk,1,"[8.905536107362e-05, 0.9999109446389264]"
900,hail mary full ov disgrace the lord has forni...,behemoth,1,"[0.00011994103138390777, 0.9998800589686161]"
701,in the forest of the eternal dreaming old oak ...,behemoth,1,"[0.00012087623001477699, 0.9998791237699852]"
1425,o serpent and lion i invoke thee inside the sh...,behemoth,1,"[0.00012513489300314262, 0.9998748651069969]"
2427,ascending a fyre a pillar ov flame on the wynd...,behemoth,1,"[0.00014687886476516532, 0.9998531211352348]"


Although *Behemoth* is only #6 in the top 10 most influential black metal bands, it owns half of the top 10 most black metal songs.  *Marduk* and *Bathory* are also represented here, but they are the only other top 10 most influential bands we see.  Interestingly, the most black metal song according to our classifier is *As the Twilight Gave Birth to the Night* by *Naglfar* (represented by 34 songs in the training dataset).

What about death metal?

In [34]:
df_correct_predictions_sorted = df_correct_predictions.sort_values(by=['probabilities'], ascending=False)
df_correct_predictions_sorted.head(10)

,lyrics,band,genre,probabilities
2034,demolition of bodies smashed and broken piles...,cannibal corpse,0,"[0.9999106901121794, 8.930988782059475e-05]"
1139,rotting alive tearing my way through the meat ...,cannibal corpse,0,"[0.9999017698571783, 9.823014282174204e-05]"
2422,under the flesh rot my lust for decayed corpse...,cannibal corpse,0,"[0.999860231573238, 0.0001397684267620009]"
1670,undead feast as they tear upon your weak fles...,cannibal corpse,0,"[0.9998086729208169, 0.00019132707918317212]"
398,imminent decay of life ruthless dead monolith...,cannibal corpse,0,"[0.9997420889892646, 0.0002579110107353983]"
2594,torsos hang from their own intestines raped of...,cannibal corpse,0,"[0.9997290216096992, 0.0002709783903007387]"
2280,profits are the powermad s motivating force ju...,dying fetus,0,"[0.9997010115464154, 0.0002989884535845336]"
2500,nocturnal demon of hate searching for his next...,suffocation,0,"[0.9994848685321369, 0.0005151314678630963]"
1223,grotesque crippled carved up carcass evil cont...,cannibal corpse,0,"[0.9994249030212643, 0.0005750969787356548]"
901,i ve seen you fighting through the time i ve f...,sepultura,0,"[0.9994028171554774, 0.0005971828445225418]"


Here *Cannibal Corpse* is the clear winner despite being #5 in the top 10 most influential death metal bands.  The only other top 10 band to show up here is *Sepultura* with the song *Lobotomy*.

## Can we do better?

While the results are pretty good and within my expectations I was of course curious if an ML model with more capacity would do better.  I played around with a few RNN setups, but the results were not better (in fact slightly worse) than the simple logistic regression.  The main problem I encountered was overfitting which I am guessing could be improved by having a larger dataset.

For the sake of completion, here's how I got to one of the better models I trained using TensorFlow (TF) and Keras.

In [6]:
import tensorflow as tf

### Step 1: Creating a TensorFlow dataset

We'll need to further split our data into train-test-validation datasets.  We'll keep the training set from before and split the test set halfway making sure to keep the proportion of death/black metal songs the same.

In [7]:
df_test, df_validation = train_test_split(df_test, test_size=0.5, stratify=df_test['genre'], random_state=1)

We then store our targets into the lists below to be later used as inputs in the TF datasets.

In [8]:
train_target = df_train.pop('genre').values.tolist()
test_target = df_test.pop('genre').values.tolist()
valid_target = df_validation.pop('genre').values.tolist()

And here are the test and validation values.

In [9]:
X_test = df_test['lyrics'].values
X_valid = df_validation['lyrics'].values

After trying out several things, I obtained the best results using the keras Tokenizer instead of CountVectorizer or TfidfVectorizer from sklearn.

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(oov_token='<UNK>')
tokenizer.fit_on_texts(X_train)

# Get our training data word index
word_index = tokenizer.word_index

# Encode training data sentences into sequences
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)
valid_sequences = tokenizer.texts_to_sequences(X_valid)

I obtained best performance by truncating the songs.

In [11]:
maxlen = 250

In [12]:
# Pad the sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
train_padded = pad_sequences(train_sequences, padding='post', maxlen=maxlen)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=maxlen)
valid_padded = pad_sequences(valid_sequences, padding='post', maxlen=maxlen)

We now have the data for the datasets.

In [13]:
ds_train = tf.data.Dataset.from_tensor_slices((train_padded, train_target))
ds_test = tf.data.Dataset.from_tensor_slices((test_padded, test_target))
ds_valid = tf.data.Dataset.from_tensor_slices((valid_padded, valid_target))

2022-02-08 18:32:55.534779: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Building the model

In [14]:
## batching 
train = ds_train.batch(20)
test = ds_test.batch(20)
valid = ds_valid.batch(20)

## feature embedding
embedding_dim = 15
vocab_size = len(word_index) + 2

In [15]:
tf.random.set_seed(1)

bi_lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        name='embed-layer'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(5, name='lstm-layer', dropout=0.5, recurrent_dropout=0.5),
        name='bidir-lstm'),
    #tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

bi_lstm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 15)          468675    
                                                                 
 dropout (Dropout)           (None, None, 15)          0         
                                                                 
 bidir-lstm (Bidirectional)  (None, 10)                840       
                                                                 
 dense (Dense)               (None, 1)                 11        
                                                                 
Total params: 469,526
Trainable params: 469,526
Non-trainable params: 0
_________________________________________________________________


In [16]:
## compile and train:
bi_lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy'])

history = bi_lstm_model.fit(
    train,
    validation_data=valid,
    epochs=10)

Epoch 1/10
302/302 [==============================] - 80s 253ms/step - loss: 0.6782 - accuracy: 0.5670 - val_loss: 0.6060 - val_accuracy: 0.7055
Epoch 2/10
302/302 [==============================] - 89s 296ms/step - loss: 0.5804 - accuracy: 0.7211 - val_loss: 0.6072 - val_accuracy: 0.6853
Epoch 3/10
302/302 [==============================] - 92s 304ms/step - loss: 0.5036 - accuracy: 0.7787 - val_loss: 0.5591 - val_accuracy: 0.7298
Epoch 4/10
302/302 [==============================] - 95s 313ms/step - loss: 0.4533 - accuracy: 0.8059 - val_loss: 0.5317 - val_accuracy: 0.7567
Epoch 5/10
302/302 [==============================] - 97s 320ms/step - loss: 0.3890 - accuracy: 0.8434 - val_loss: 0.5536 - val_accuracy: 0.7480
Epoch 6/10
302/302 [==============================] - 94s 311ms/step - loss: 0.3379 - accuracy: 0.8663 - val_loss: 0.5634 - val_accuracy: 0.7668
Epoch 7/10
302/302 [==============================] - 91s 301ms/step - loss: 0.2967 - accuracy: 0.8858 - val_loss: 0.6414 - val_ac

In [17]:
## evaluate on the test data
test_results = bi_lstm_model.evaluate(test)
print('Test Acc.: {:.2f}%'.format(test_results[1]*100))

75/75 [==============================] - 3s 33ms/step - loss: 0.7607 - accuracy: 0.7446
Test Acc.: 74.46%


As mentioned before, the performance is not better than the simple logistic regression and there are clear signs of overfitting.  I have also tried eliminating stopwords and adding more/fewer layers but the performance was remarkably consistent: whenever the model learns, it can get up to high 90s on the training set, but on validation and test sets I have not managed to make it do better than 74-76% accuracy.

## Conclusions and ToDos

Does ok, would like to have a bigger dataset. Exciting that bands not represented by many songs in the training dataset can be correctly labelled.  Also that the wrongly labelled ones are lyrics that a human might also struggle labelling correctly without also hearing the music, or where the labelling is in some sense not correct - bands do change direction or are anyway not entirely within one genre or another.

Would also be cool to try to distinguish between more metal genres and also non-metal songs.


In [1]:
import os
os.system('jupyter nbconvert --to html metal-classification.ipynb')

[NbConvertApp] Converting notebook metal-classification.ipynb to html
[NbConvertApp] Writing 1190590 bytes to metal-classification.html


0